In [1]:
#利用pytorch来进行优化;
import numpy as np
import h5py
from skimage import io
%matplotlib inline
from matplotlib import pyplot as plt
import torch
from torch.autograd import Variable
import torch.optim as optim

In [ ]:
from __future__ import print_function

In [12]:
#优化实验!
x = Variable(torch.ones(1), requires_grad=True)
y = Variable(torch.ones(1), requires_grad=True)
z=2*x**2+3*y**3
z.backward(retain_variables=True)
learning_rate = 0.1
x.data.sub_(learning_rate*x.grad.data)
x.grad.data=torch.FloatTensor([0.0])
z.backward(retain_variables=True)
#w1.data -= learning_rate * w1.grad.data
x.data.sub_(learning_rate*x.grad.data)
#手动更新也可行;

x = Variable(torch.ones(1), requires_grad=True)
y = Variable(torch.ones(1), requires_grad=True)
z=2*x**2+3*y**3
#使用pytorch中的optim优化器;
optimizer = optim.SGD([x,y], lr = 0.01)
optimizer.zero_grad() # zero the gradient buffers，必须要置零
z.backward(retain_variables=True)
optimizer.step() # Does the update
#可行！

In [ ]:
#variable:s_mean;alpha;s;S;f;P;R;t;U_proj;d;U_label;sigma;
f=h5py.File('shapemodel.h5','r')
fly=open('testface_landmarks.txt')
ftxt=fly.readlines()
for i in range(len(ftxt)):
    ftxt[i]=ftxt[i].strip('\n').split(' ')
for i in range(68):
    for j in range(2):
        ftxt[i][j]=float(ftxt[i][j])
U_label=Variable(torch.FloatTensor(np.array(ftxt).T))

d=f['keypoints'][:][0]
s_mean=Variable(torch.FloatTensor(f['mean_shape'][:]))   #159645*1;
s=Variable(torch.FloatTensor(f['pca_basis'][:]))    #159645*50;
P=Variable(torch.FloatTensor(np.array([[1,0,0],[0,1,0]])))
sigma=Variable(torch.FloatTensor(f['sigma'][:]))   #50*1;

alpha=Variable(torch.rand((50,1)),requires_grad=True)
S=s_mean+s.mm(alpha)
S=S.view(53215,3)
f=Variable(torch.ones((2,3)),requires_grad=True)
R=Variable(torch.rand((3,3)),requires_grad=True)
t=Variable(torch.rand(2),requires_grad=True)
S_final=Variable(torch.rand(3,68))
for i in range(68):
    S_final[:,i]=S[d[i]]
U_proj=(f*P).mm(R).mm(S_final)
for i in range(68):
    U_proj[:,i]=U_proj[:,i]+t
loss2=torch.sum((alpha*alpha)/(sigma*sigma))
loss1=torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
loss=loss1+loss2

In [15]:
learning_rate=0.0005
loss_100=0
loss_print=[]
loss_new=0
for i in range(10000):
    #loss_old=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
    f.grad.data=torch.FloatTensor([[0.0,0.0,0.0],[0.0,0.0,0.0]])
    R.grad.data=torch.zeros((3,3))
    t.grad.data=torch.zeros((2,1))
    alpha.grad.data=torch.zeros((50,1))
    loss.backward(retain_variables=True)
    #w1.data -= learning_rate * w1.grad.data
    f.data.sub_(learning_rate*f.grad.data)
    R.data.sub_(learning_rate*R.grad.data)
    t.data.sub_(learning_rate*t.grad.data)
    alpha.data.sub_(learning_rate*alpha.grad.data)    
    loss_new=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
    loss_100=loss_100+loss_new
    if i%100==0:
        loss_print.append(loss_100/100.0)
        loss_100=0
    if i%1000==0:
        print i

stop=1
while stop>0:
    #loss_old=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
    f.grad.data=torch.FloatTensor([[0.0,0.0,0.0],[0.0,0.0,0.0]])
    R.grad.data=torch.zeros((3,3))
    t.grad.data=torch.zeros((2,1))
    alpha.grad.data=torch.zeros((50,1))
    loss.backward(retain_variables=True)
    #w1.data -= learning_rate * w1.grad.data
    f.data.sub_(learning_rate*f.grad.data)
    R.data.sub_(learning_rate*R.grad.data)
    t.data.sub_(learning_rate*t.grad.data)
    alpha.data.sub_(learning_rate*alpha.grad.data)
    loss_new=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
    loss_100=loss_100+loss_new.data
    if i%100==0:
        loss_print.append(loss_100/100.0)
        loss_100=0
        if len(loss_print)>2 and torch.abs(loss_print[len(loss_print)-1]-loss_print[len(loss_print)-2])<0.1:
            stop=0
    if i%1000==0:
        print i

In [ ]:
optimizer = optim.SGD([f,R,t,alpha], lr = 0.001)
loss_100=0
loss_print=[]
loss_new=0
stop=1
while stop>0:
    #loss_old=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
    optimizer.zero_grad() # zero the gradient buffers，必须要置零
    loss.backward(retain_variables=True)
    optimizer.step() # Does the update
    loss_new=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
    loss_100=loss_100+loss_new.data
    if i%100==0:
        loss_print.append(loss_100/100.0)
        loss_100=0
        if len(loss_print)>2 and torch.abs(loss_print[len(loss_print)-1]-loss_print[len(loss_print)-2])<0.1:
            stop=0
    if i%1000==0:
        print i

In [307]:
from __future__ import print_function
for i in range(50):
    print('%f' % alpha.data.numpy()[i],end=' ')
    #print(' ')

0.228512 0.414567 0.452815 0.484734 0.092824 0.238131 0.947343 0.195758 0.773634 0.910061 0.418036 0.186317 0.224193 0.967962 0.364416 0.705933 0.078815 0.705428 0.483994 0.168831 0.370696 0.721676 0.638648 0.039472 0.778362 0.893754 0.354865 0.246894 0.216938 0.894282 0.563633 0.351977 0.099967 0.660270 0.582316 0.510889 0.898550 0.362596 0.986510 0.959051 0.743724 0.804989 0.059532 0.871163 0.170256 0.035645 0.679062 0.528763 0.242468 0.172112 

In [ ]:
#提交时对格式有要求:注意print出来的格式，有时候看着在一行但是实际有换行符，就会提交不了;
#对提交的结果格式解析时没有强制要求alpha的和为1;

In [71]:
import numpy as np
import h5py
from skimage import io
from matplotlib import pyplot as plt
import torch
from torch.autograd import Variable
import torch.optim as optim

#variable:s_mean;alpha;s;S;f;P;R;t;U_proj;d;U_label;sigma;
f=h5py.File('shapemodel.h5','r')
fly=open('testface_landmarks.txt')
ftxt=fly.readlines()
for i in range(len(ftxt)):
    ftxt[i]=ftxt[i].strip('\n').split(' ')
for i in range(68):
    for j in range(2):
        ftxt[i][j]=float(ftxt[i][j])
U_label=Variable(torch.FloatTensor(np.array(ftxt).T))

d=f['keypoints'][:][0]
s_mean=Variable(torch.FloatTensor(f['mean_shape'][:]))   #159645*1;
s=Variable(torch.FloatTensor(f['pca_basis'][:]))    #159645*50;
P=Variable(torch.FloatTensor(np.array([[1,0,0],[0,1,0]])))
sigma=Variable(torch.FloatTensor(f['sigma'][:]))   #50*1;

alpha=Variable(torch.rand((50,1)),requires_grad=True)
S=s_mean+s.mm(alpha)
S=S.view(53215,3)
f=Variable(torch.ones((2,3)),requires_grad=True)
R=Variable(torch.rand((3,3)),requires_grad=True)
t=Variable(torch.rand((2)),requires_grad=True)
S_final=Variable(torch.rand(3,68))
for i in range(68):
    S_final[:,i]=S[d[i]]
U_proj=(f*P).mm(R).mm(S_final)
for i in range(68):
    U_proj[:,i]=U_proj[:,i]+t
loss2=torch.sum((alpha*alpha)/(sigma*sigma))
loss1=torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
loss=loss1+loss2

optimizer = optim.SGD([f,R,t,alpha], lr = 0.0001)
loss_100=0
loss_print=[]
loss_new=0
#stop=1
#while stop>0:
for i in range(1000):
    #loss_old=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
    optimizer.zero_grad() # zero the gradient buffers
    loss.backward(retain_variables=True)
    optimizer.step() # Does the update
    loss_new=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
    loss_100=loss_100+loss_new.data.numpy()[0]
    if i%10==0:
        print loss_100/100.0
        loss_print.append(loss_100/100.0)
        loss_100=0
        #if len(loss_print)>2 and torch.abs(loss_print[len(loss_print)-1]-loss_print[len(loss_print)-2])<0.1:
           # stop=0
    if i%10==0:
        print i
#f_out=f.data.view(6).numpy()
#R_out=R.data.view(9).numpy()
#t_out=t.data.view(2).numpy()
#alpha_out=alpha.data.view(50).numpy()
#np.savetxt('f.txt',f_out,fmt='%.4f')
#np.savetxt('R.txt',R_out,fmt='%.4f')
#np.savetxt('t.txt',t_out,fmt='%.4f')
#np.savetxt('alpha.txt',alpha_out,fmt='%.4f')
#np.savetxt('loss.txt',loss_print,fmt='%.4f')

6445.176875
0


RuntimeError: Trying to backward through the graph second time, but the buffers have already been freed. Please specify retain_variables=True when calling backward for the first time.

In [74]:
f=Variable(torch.ones((2,3)),requires_grad=True)
ff=torch.ones(2,3)


In [ ]:
#fmt控制输出格式和列之间的分隔符，其中的设置次数要和列数对应；newline对应不同行之间的分隔符;
#大小不同的数组好像不能同时读入，也没有办法不覆盖在文件末尾追加!
np.savetxt('1.txt',(ly,sr),fmt='%.4f %.4f %.4f %.4f',newline=' ')
ly=np.array([1,2])
sr=np.array([[5,6,7,8]])
np.savetxt('3.txt',(ly,sr),fmt='%.4f')

this is the part for try!

In [2]:
import numpy as np
import h5py
from skimage import io
from matplotlib import pyplot as plt
import torch
from torch.autograd import Variable
import torch.optim as optim

In [1]:
#variable:s_mean;alpha;s;S;f;P;R;t;U_proj;d;U_label;sigma;
f=h5py.File('shapemodel.h5','r')
fly=open('testface_landmarks.txt')
ftxt=fly.readlines()
for i in range(len(ftxt)):
    ftxt[i]=ftxt[i].strip('\n').split(' ')
for i in range(68):
    for j in range(2):
        ftxt[i][j]=float(ftxt[i][j])
U_label=Variable(torch.FloatTensor(np.array(ftxt).T))

d=f['keypoints'][:][0]
s_mean=Variable(torch.FloatTensor(f['mean_shape'][:]))   #159645*1;
s=Variable(torch.FloatTensor(f['pca_basis'][:]))    #159645*50;
P=Variable(torch.FloatTensor(np.array([[1,0,0],[0,1,0]])))
sigma=Variable(torch.FloatTensor(f['sigma'][:]))   #50*1;

alpha=Variable(torch.rand((50,1)),requires_grad=True)
S=s_mean+s.mm(alpha)
S=S.view(53215,3)
f=Variable(torch.ones((2,3)),requires_grad=True)
R=Variable(torch.rand((3,3)),requires_grad=True)
t=Variable(torch.rand((2)),requires_grad=True)
S_final=Variable(torch.rand(3,68))
for i in range(68):
    S_final[:,i]=S[d[i]]
U_proj=(f*P).mm(R).mm(S_final)
for i in range(68):
    U_proj[:,i]=U_proj[:,i]+t
loss2=torch.sum((alpha*alpha)/(sigma*sigma))
loss1=torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
loss=loss1+loss2

optimizer = optim.SGD([f,R,t,alpha], lr = 0.0001)
loss_100=0
loss_print=[]
loss_new=0

In [107]:
#loss_old=torch.sum((alpha*alpha)/(sigma*sigma))+torch.sqrt(torch.sum((U_proj-U_label)*(U_proj-U_label)))
optimizer.zero_grad() # zero the gradient buffers
loss.backward(retain_variables=True)
optimizer.step() # Does the update

fry for why there is no gradient?

In [54]:
x = Variable(torch.rand((2,2)), requires_grad=True)
y = Variable(torch.rand((2,2)), requires_grad=True)
ly=torch.norm(x-y)
optimizer = optim.SGD([x,y], lr = 0.05)
print (x,y,ly)

(Variable containing:
 0.2840  0.7693
 0.7379  0.2060
[torch.FloatTensor of size 2x2]
, Variable containing:
 0.0928  0.4751
 0.9091  0.1095
[torch.FloatTensor of size 2x2]
, Variable containing:
 0.4022
[torch.FloatTensor of size 1]
)


In [100]:
x = Variable(torch.Tensor([[1,2],[3,4]]), requires_grad=True)
y = Variable(torch.Tensor([[8,7],[6,5]]), requires_grad=True)
z=x-y
z1=z*z
ly=z1.mean()
optimizer = optim.SGD([x,y], lr = 0.05)
print (x,y,z1,ly)

(Variable containing:
 1  2
 3  4
[torch.FloatTensor of size 2x2]
, Variable containing:
 8  7
 6  5
[torch.FloatTensor of size 2x2]
, Variable containing:
 49  25
  9   1
[torch.FloatTensor of size 2x2]
, Variable containing:
 21
[torch.FloatTensor of size 1]
)


In [101]:
ly.backward()
optimizer.step()
print(x.grad,y.grad,x,y,ly)

(Variable containing:
-3.5000 -2.5000
-1.5000 -0.5000
[torch.FloatTensor of size 2x2]
, Variable containing:
 3.5000  2.5000
 1.5000  0.5000
[torch.FloatTensor of size 2x2]
, Variable containing:
 1.1750  2.1250
 3.0750  4.0250
[torch.FloatTensor of size 2x2]
, Variable containing:
 7.8250  6.8750
 5.9250  4.9750
[torch.FloatTensor of size 2x2]
, Variable containing:
 21
[torch.FloatTensor of size 1]
)


In [102]:
optimizer.zero_grad()

In [95]:
x=x
y=y
z=x-y
z1=z*z
ly=z1.mean()
print(x,y,ly)

(Variable containing:
 1.1750  2.1250
 3.0750  4.0250
[torch.FloatTensor of size 2x2]
, Variable containing:
 7.8250  6.8750
 5.9250  4.9750
[torch.FloatTensor of size 2x2]
, Variable containing:
 18.9525
[torch.FloatTensor of size 1]
)


In [103]:
ly.backward()
optimizer.step()
print(x.grad,y.grad,x,y,ly)

RuntimeError: Trying to backward through the graph second time, but the buffers have already been freed. Please specify retain_variables=True when calling backward for the first time.

try for another problem

In [9]:
x=Variable(torch.rand((2,2)),requires_grad=True)
y=Variable(torch.rand(1),requires_grad=True)
z=Variable(torch.rand((2,2)))

In [10]:
for i in range(2):
    for j in range(2):
        z[i,j]=x[i,j]+y

In [13]:
loss=(z*z).mean()

In [15]:
loss.backward()

In [16]:
x.grad

Variable containing:
 0.7061  0.4787
 0.5120  0.5318
[torch.FloatTensor of size 2x2]

In [18]:
y.grad

Variable containing:
 2.2287
[torch.FloatTensor of size 1]

In [19]:
print(x,y)

(Variable containing:
 0.5414  0.0866
 0.1533  0.1928
[torch.FloatTensor of size 2x2]
, Variable containing:
 0.8708
[torch.FloatTensor of size 1]
)


In [26]:
sum(sum(x))*0.5+2*y

Variable containing:
 2.2287
[torch.FloatTensor of size 1]